# PART 1: Data Retrieval

## This script contains the following steps:

### (1) Data retrieval using Reddit API 

A customisable function that utilises the PRAW Reddit API wrapper (1) to retrieve the maximum number of posts in the subreddit r/news containing the term "immigr" in the title. Due to the limit set by Reddit, the maximum number of posts I am able to retrieve is 250 (a request maximally yields the first 250 posts). Then, the maximum number of comments replying directly to the post ("top-level comments") are retrieved (max. 200 per post, as default limit by Reddit). 
  
  (1)https://praw.readthedocs.io/en/stable/

### (2) Data extraction from archived data files

As previous ways to retrieve data other than the first 250 posts (primarily pushshift API) were recently disabled, the data is enriched using existing archived posts and comments from r/news, downloaded from (1). As these files are very large, the script (taken and adjusted from 2) generates a post file filtered for the time range (between 2016 and 2022) and matching the "immigr" keyword. Next, the IDs of these posts are utilised to retrieve matching replies from the comment file.
  
  (1)https://academictorrents.com/details/c398a571976c78d346c325bd75c47b82edf6124e/tech  
    (2) https://github.com/Watchful1/PushshiftDumps/blob/master/scripts/filter_file.py

# Import required packages

In [1]:
import praw
import pandas as pd
import datetime

In [2]:
import regex
import re

In [3]:
import zstandard
import os
import json
import sys
import csv
from datetime import datetime
import logging.handlers

# Reddit API Data Retrieval using PRAW:

In [4]:
#Create Reddit Instance
reddit = praw.Reddit(client_id = "FnbCy6YOk4XXaffasw6QOg",
                     client_secret = "4A79GisTLrk0PqlLkacML-tFXP-44A",
                     user_agent = "bdca_finalproject_ms",
                     username = "bdca_project_acc",
                     password = "bdca_reddit_account")

### Parameters:

In [5]:
# set limit for maximum number of posts 
maxposts = 250

#specify subreddit 
subreddit = reddit.subreddit("news")

#specify searchterm for desired topic
searchterm = "immigr"

# set limit for "replace more" (i.e., how often the end of the page should be "clicked" to retrieve more comments)
rm_limit = 0

# specify by what comment should be sorted (e.g., "best", "controversial", etc.)
comment_sort_logic = "best"

### Create separate dictionaries to store posts and comments

**Posts**

In [6]:
# create dictionary where each key refers to a list into which retrieved data is passed
post_dict = {"post_id":[],
         "title": [], 
        "date": []}

**Comments**

In [7]:
# create comments dictionary
comments_dict = {"post_id":[], 
                "comment_id":[],
                "comment":[], 
                "date":[]}

### Function for Data Retrieval

In [8]:
def get_reddit_data (sub):
    for post in subreddit.search(searchterm, limit=maxposts): 
        
        # (1) Retrieve post metadata
        post_dict["post_id"].append(post.id), # id of post
        post_dict["title"].append(post.title), # post title
        post_dict["date"].append(post.created_utc), #date postd
        
        # (2) Retrieve comments and their metadata for these posts
        post.comment_sort = comment_sort_logic
        post.comments.replace_more(limit=rm_limit)
        for top_level_comment in post.comments:
            comments_dict["post_id"].append(top_level_comment.link_id), # id of post
            comments_dict["comment_id"].append(top_level_comment.id), # id of the top-level comment (uniques)
            comments_dict["comment"].append(top_level_comment.body) # body of the comment      

### Data Retrieval:

In [9]:
# apply function to specified subreddit 
get_reddit_data(subreddit)

### Convert Retrieved Data to Dataframes & basic cleaning steps

### (1) Posts:

In [10]:
posts_df = pd.DataFrame(post_dict)
print(f"{len(posts_df)} posts were downloaded")

215 posts were downloaded


In [11]:
# inspect data frame
posts_df.head()

,post_id,title,date
0,zf28mm,Nearly 1 million immigrants became U.S. citize...,1.670423e+09
1,vqpw0n,California set to become first state to provid...,1.656879e+09
2,yl1psd,Immigration officials say Pelosi attacker was ...,1.667479e+09
3,ur40rc,Authorities: Gunman in deadly attack at Califo...,1.652730e+09
4,x6fx61,Farmers now pushing for immigration reform to ...,1.662384e+09


#### Change Date Formatting to have YYYY-MM-DD

In [12]:
# change utc date format to human readable
posts_df["date"] = pd.to_datetime(posts_df["date"],unit="s")

In [13]:
posts_df["date"] = posts_df["date"].dt.strftime('%Y-%m-%d')

In [14]:
posts_df.head()

,post_id,title,date
0,zf28mm,Nearly 1 million immigrants became U.S. citize...,2022-12-07
1,vqpw0n,California set to become first state to provid...,2022-07-03
2,yl1psd,Immigration officials say Pelosi attacker was ...,2022-11-03
3,ur40rc,Authorities: Gunman in deadly attack at Califo...,2022-05-16
4,x6fx61,Farmers now pushing for immigration reform to ...,2022-09-05


In [15]:
# Save as .csv file
posts_df.to_csv("news_immi_posts.csv")

### (2) Comments:

In [16]:
comments_df = pd.DataFrame(comments_dict)
print(f"{len(comments_df)} comments were downloaded")

8714 comments were downloaded


#### Remove post_id-prefix
When retrieving the comments data from reddit, the post_id is always returned with an additional "t3_" before it. Using a regular expression, this is removed.

In [20]:
# remove t3_
comments_df["post_id"] = comments_df["post_id"].str.replace(r"^t3_", "", regex =True)

In [22]:
# save as .csv file 
comments_df.to_csv('news_immi_comments.csv')

# Get Archived Data from r/news

The following code searches a .zst file for the specified date range and key word, and generates the posts file, saving it as csv. This procedure followed the steps outlined in (1), code for data extraction was slightly adapted but followed the script in (2)
  
  (1) https://old.reddit.com/r/pushshift/comments/11ef9if/separate_dump_files_for_the_top_20k_subreddits/ 
  
  (2) https://github.com/Watchful1/PushshiftDumps/blob/master/scripts/filter_file.py

### Get all submissions from r/news 2016–2022 containing "immigr" in the title:

In [23]:
# input file name 
input_file = "news_submissions.zst"

# name to the output file, output format
output_file = "news_submissions_filtered"
output_format = "csv"
is_submission = "submission" in input_file

# only output items between these two dates
from_date = datetime.strptime("2016-01-01", "%Y-%m-%d")
to_date = datetime.strptime("2022-12-31", "%Y-%m-%d")

# return only objects where the title contains either "immigration", "immigrant", "immigrants"
field = "title"
values = ["immigration", "immigrant", "immigrants"]
exact_match = False

# sets up logging to the console as well as a file
log = logging.getLogger("bot")
log.setLevel(logging.INFO)
log_formatter = logging.Formatter('%(asctime)s - %(levelname)s: %(message)s')
log_str_handler = logging.StreamHandler()
log_str_handler.setFormatter(log_formatter)
log.addHandler(log_str_handler)
if not os.path.exists("logs"):
	os.makedirs("logs")
log_file_handler = logging.handlers.RotatingFileHandler(os.path.join("logs", "bot.log"), maxBytes=1024*1024*16, backupCount=5)
log_file_handler.setFormatter(log_formatter)
log.addHandler(log_file_handler)

# function to write to zst
def write_line_zst(handle, line):
	handle.write(line.encode('utf-8'))
	handle.write("\n".encode('utf-8'))

# function to write to json
def write_line_json(handle, obj):
	handle.write(json.dumps(obj))
	handle.write("\n")

# function to write to csv
def write_line_csv(writer, obj, is_submission):
	output_list = []
	output_list.append(str(obj['id']))
	if is_submission:
		output_list.append(obj['title'])
	output_list.append(datetime.fromtimestamp(obj['created_utc']).strftime("%Y-%m-%d"))
	writer.writerow(output_list)


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		log.info(f"Decoding error with {bytes_read:,} bytes, reading another chunk")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)

			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line.strip(), file_handle.tell()

			buffer = lines[-1]

		reader.close()


if __name__ == "__main__":
	output_path = f"{output_file}.{output_format}"

	writer = None
	if output_format == "zst":
		handle = zstandard.ZstdCompressor().stream_writer(open(output_path, 'wb'))
	elif output_format == "txt":
		handle = open(output_path, 'w', encoding='UTF-8')
	elif output_format == "csv":
		handle = open(output_path, 'w', encoding='UTF-8', newline='')
		writer = csv.writer(handle)
	else:
		log.error(f"Unsupported output format {output_format}")
		sys.exit()

	values = [value.lower() for value in values]  # convert to lowercase

	file_size = os.stat(input_file).st_size
	file_bytes_processed = 0
	created = None
	matched_lines = 0
	bad_lines = 0
	total_lines = 0
	for line, file_bytes_processed in read_lines_zst(input_file):
		total_lines += 1
		if total_lines % 100000 == 0:
			log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {total_lines:,} : {matched_lines:,} : {bad_lines:,} : {file_bytes_processed:,}:{(file_bytes_processed / file_size) * 100:.0f}%")

		try:
			obj = json.loads(line)
			created = datetime.utcfromtimestamp(int(obj['created_utc']))

			if created < from_date:
				continue
			if created > to_date:
				continue

			field_value = obj[field].lower()
			matched = False
			for value in values:
				if exact_match:
					if value == field_value:
						matched = True
						break
				else:
					if value in field_value:
						matched = True
						break
			if not matched:
				continue

			matched_lines += 1
			if output_format == "zst":
				write_line_zst(handle, line)
			elif output_format == "csv":
				write_line_csv(writer, obj, is_submission)
			elif output_format == "txt":
				write_line_json(handle, obj)
		except (KeyError, json.JSONDecodeError) as err:
			bad_lines += 1

	handle.close()
	log.info(f"Complete : {total_lines:,} : {matched_lines:,} : {bad_lines:,}")

2023-05-27 02:01:34,534 - INFO: 2011-06-01 22:49:52 : 100,000 : 0 : 0 : 11,141,375:1%
2023-05-27 02:01:37,553 - INFO: 2012-06-22 19:02:11 : 200,000 : 0 : 0 : 28,574,350:3%
2023-05-27 02:01:40,495 - INFO: 2013-02-27 07:34:56 : 300,000 : 0 : 0 : 46,531,625:6%
2023-05-27 02:01:42,887 - INFO: 2013-07-08 21:45:08 : 400,000 : 0 : 0 : 63,964,600:8%
2023-05-27 02:01:45,605 - INFO: 2013-10-07 14:04:50 : 500,000 : 0 : 0 : 63,964,600:8%
2023-05-27 02:01:50,182 - INFO: 2013-12-27 08:18:14 : 600,000 : 0 : 0 : 81,397,575:10%
2023-05-27 02:01:52,394 - INFO: 2014-03-12 04:21:20 : 700,000 : 0 : 0 : 98,961,625:12%
2023-05-27 02:01:54,299 - INFO: 2014-05-25 05:34:43 : 800,000 : 0 : 0 : 116,394,600:14%
2023-05-27 02:01:55,737 - INFO: 2014-08-11 23:09:15 : 900,000 : 0 : 0 : 116,394,600:14%
2023-05-27 02:01:57,642 - INFO: 2014-10-20 06:01:06 : 1,000,000 : 0 : 0 : 134,089,725:16%
2023-05-27 02:01:59,578 - INFO: 2015-01-01 05:52:06 : 1,100,000 : 0 : 0 : 151,784,850:18%
2023-05-27 02:02:01,571 - INFO: 2015-03-

In [24]:
# Loading the extracted file: 
archived_posts = pd.read_csv('news_submissions_filtered.csv', header=None)

In [25]:
# Inspecting file
archived_posts.head()

,0,1,2
0,3yzvng,When Your World Comes Apart: An Immigrant's Story,2016-01-01
1,3z0jn8,No-Go Zones: German Police Admit To Losing Con...,2016-01-01
2,3z0qa9,Amazon are giving proceeds of a German far rig...,2016-01-01
3,3z13lf,3 new areas to house young immigrants - The U....,2016-01-01
4,3zakcu,Immigrant kids’ needs go beyond bilingual clas...,2016-01-03


In [26]:
# adding a header column 
header = ['post_id', 'title', 'date']
archived_posts.columns = header

In [27]:
archived_posts.head()

,post_id,title,date
0,3yzvng,When Your World Comes Apart: An Immigrant's Story,2016-01-01
1,3z0jn8,No-Go Zones: German Police Admit To Losing Con...,2016-01-01
2,3z0qa9,Amazon are giving proceeds of a German far rig...,2016-01-01
3,3z13lf,3 new areas to house young immigrants - The U....,2016-01-01
4,3zakcu,Immigrant kids’ needs go beyond bilingual clas...,2016-01-03


In [28]:
print(f"{len(archived_posts)} posts between 2016-01-01 and 2022-12-31 were downloaded")

11446 posts between 2016-01-01 and 2022-12-31 were downloaded


## Create final posts dataframe

In [29]:
# Appending the scraped posts (posts_df) to the downloaded posts from the archive (archived_posts)
# and saving it as new object "allposts", maintaining a continuous index.
allposts = archived_posts.append(posts_df, ignore_index = True)

/var/folders/2w/bw2mbvzn6ns_37v4dn94xthm0000gn/T/ipykernel_20288/2084607609.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  allposts = archived_posts.append(posts_df, ignore_index = True)


In [30]:
# drop posts that appear twice: 
allposts = allposts.drop_duplicates(subset=["post_id"])

In [31]:
print(f"the data frame with all posts contains {len(allposts)} posts with *immigr* in the title")

the final data frame contains 11477 with *immigr* in the title


### Shuffle data & create reduced data frame: 
After shuffling the data, a random sample of N=6000 posts is drawn. This is not a necessary step in case there are no space or time constraints. However, for this proof of concept, the sample of posts was reduced to generate a sizeable but still manageable comment file, as all top-level comments to a post will be extracted using post ids. If a larger N for both comments and posts is desired, the "posts_n" parameter can be adjusted

In [32]:
# shuffle the data, such that it returns all rows in random order, setting random state seed so same shuffling 
# when rerunning script
allposts = allposts.sample(frac=1, random_state = 100)

In [33]:
# set deisred sample size
post_n = 6000

In [34]:
# draw random sample
posts_red = allposts.sample(post_n, random_state=99)

In [35]:
# create csv with reduced posts 
posts_red.to_csv("posts_red.csv")

In [49]:
# create txt file with post ids to retrieve only top-level comments for the corresponding post_ids
posts_red_id_s = posts_red["post_id"].astype(str)
with open("posts_red_id_s.txt", "w") as file:
    file.write('\n'.join(posts_red_id_s))

## Get all top-level comments (direct replies to a post) for the posts in the reduced data frame:

In [74]:
# put the path to the input file
input_file = "news_comments.zst"

# put the name or path to the output file. The file extension from below will be added automatically
output_file = "news_comments_id_filtered"
output_format = "csv"
is_submission = "comment" in input_file
single_field = None 
values_file = "posts_red_id_s.txt"
field = "parent_id"  # or link_id -- in the comment object, this is the field that contains the submission id
exact_match = False

# sets up logging to the console as well as a file
log = logging.getLogger("bot")
log.setLevel(logging.INFO)
log_formatter = logging.Formatter('%(asctime)s - %(levelname)s: %(message)s')
log_str_handler = logging.StreamHandler()
log_str_handler.setFormatter(log_formatter)
log.addHandler(log_str_handler)
if not os.path.exists("logs"):
	os.makedirs("logs")
log_file_handler = logging.handlers.RotatingFileHandler(os.path.join("logs", "bot.log"), maxBytes=1024*1024*16, backupCount=5)
log_file_handler.setFormatter(log_formatter)
log.addHandler(log_file_handler)


def write_line_zst(handle, line):
	handle.write(line.encode('utf-8'))
	handle.write("\n".encode('utf-8'))


def write_line_json(handle, obj):
	handle.write(json.dumps(obj))
	handle.write("\n")


#def write_line_single(handle, obj, field):
#	if field in obj:
#		handle.write(obj[field])
#	else:
#		log.info(f"{field} not in object {obj['id']}")
#	handle.write("\n")

    
def write_line_csv(writer, obj, is_submission):
	output_list = []
	output_list.append(str(obj['parent_id']))
	output_list.append(str(obj['id']))
	if is_submission:
		output_list.append(obj['body'])
#	output_list.append(datetime.fromtimestamp(obj['created_utc']).strftime("%Y-%m-%d"))
	writer.writerow(output_list)


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		log.info(f"Decoding error with {bytes_read:,} bytes, reading another chunk")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)

			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line.strip(), file_handle.tell()

			buffer = lines[-1]

		reader.close()


if __name__ == "__main__":
	if single_field is not None:
		log.info("Single field output mode, changing output file format to txt")
		output_format = "txt"
	output_path = f"{output_file}.{output_format}"

	writer = None
	if output_format == "zst":
		log.info("Output format set to zst")
		handle = zstandard.ZstdCompressor().stream_writer(open(output_path, 'wb'))
	elif output_format == "txt":
		log.info("Output format set to txt")
		handle = open(output_path, 'w', encoding='UTF-8')
	elif output_format == "csv":
		log.info("Output format set to csv")
		handle = open(output_path, 'w', encoding='UTF-8', newline='')
		writer = csv.writer(handle)
	else:
		log.error(f"Unsupported output format {output_format}")
		sys.exit()

	if values_file is not None:
		values = []
		with open(values_file, 'r') as values_handle:
			for value in values_handle:
				values.append(value.strip().lower())
		log.info(f"Loaded {len(values)} from values file")
	else:
		values = [value.lower() for value in values]  # convert to lowercase

	file_size = os.stat(input_file).st_size
	file_bytes_processed = 0
	created = None
	matched_lines = 0
	bad_lines = 0
	total_lines = 0
	for line, file_bytes_processed in read_lines_zst(input_file):
		total_lines += 1
		if total_lines % 100000 == 0:
			log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {total_lines:,} : {matched_lines:,} : {bad_lines:,} : {file_bytes_processed:,}:{(file_bytes_processed / file_size) * 100:.0f}%")

		try:
			obj = json.loads(line)
			created = datetime.utcfromtimestamp(int(obj['created_utc']))

#			if created < from_date:
#				continue
#			if created > to_date:
#				continue

			field_value = obj[field].lower()
			matched = False
			for value in values:
				if exact_match:
					if value == field_value:
						matched = True
						break
				else:
					if value in field_value:
						matched = True
						break
			if not matched:
				continue

			matched_lines += 1
			if output_format == "zst":
				write_line_zst(handle, line)
			elif output_format == "csv":
				write_line_csv(writer, obj, is_submission)
			elif output_format == "txt":
				if single_field is not None:
					write_line_single(handle, obj, single_field)
				else:
					write_line_json(handle, obj)
		except (KeyError, json.JSONDecodeError) as err:
			bad_lines += 1
	handle.close()
	log.info(f"Complete : {total_lines:,} : {matched_lines:,} : {bad_lines:,}")

2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:14,996 - INFO: Output format set to csv
2023-05-27 03:55:15,035 - INFO: Loaded 6000 from values file
2023-05-27 03:55:15,035 - INFO: Loaded 6000 from values file
2023-05-27 03:55:15,035

2023-05-27 04:01:25,348 - INFO: 2012-04-25 18:38:47 : 600,000 : 76 : 0 : 73,139,850:1%
2023-05-27 04:01:25,348 - INFO: 2012-04-25 18:38:47 : 600,000 : 76 : 0 : 73,139,850:1%
2023-05-27 04:01:25,348 - INFO: 2012-04-25 18:38:47 : 600,000 : 76 : 0 : 73,139,850:1%
2023-05-27 04:01:25,348 - INFO: 2012-04-25 18:38:47 : 600,000 : 76 : 0 : 73,139,850:1%
2023-05-27 04:01:25,348 - INFO: 2012-04-25 18:38:47 : 600,000 : 76 : 0 : 73,139,850:1%
2023-05-27 04:01:25,348 - INFO: 2012-04-25 18:38:47 : 600,000 : 76 : 0 : 73,139,850:1%
2023-05-27 04:01:25,348 - INFO: 2012-04-25 18:38:47 : 600,000 : 76 : 0 : 73,139,850:1%
2023-05-27 04:01:25,348 - INFO: 2012-04-25 18:38:47 : 600,000 : 76 : 0 : 73,139,850:1%
2023-05-27 04:01:25,348 - INFO: 2012-04-25 18:38:47 : 600,000 : 76 : 0 : 73,139,850:1%
2023-05-27 04:01:25,348 - INFO: 2012-04-25 18:38:47 : 600,000 : 76 : 0 : 73,139,850:1%
2023-05-27 04:01:25,348 - INFO: 2012-04-25 18:38:47 : 600,000 : 76 : 0 : 73,139,850:1%
2023-05-27 04:01:25,348 - INFO: 2012-04-25 

2023-05-27 04:07:33,607 - INFO: 2013-03-09 17:28:51 : 1,200,000 : 171 : 0 : 143,920,350:2%
2023-05-27 04:07:33,607 - INFO: 2013-03-09 17:28:51 : 1,200,000 : 171 : 0 : 143,920,350:2%
2023-05-27 04:07:33,607 - INFO: 2013-03-09 17:28:51 : 1,200,000 : 171 : 0 : 143,920,350:2%
2023-05-27 04:07:33,607 - INFO: 2013-03-09 17:28:51 : 1,200,000 : 171 : 0 : 143,920,350:2%
2023-05-27 04:07:33,607 - INFO: 2013-03-09 17:28:51 : 1,200,000 : 171 : 0 : 143,920,350:2%
2023-05-27 04:07:33,607 - INFO: 2013-03-09 17:28:51 : 1,200,000 : 171 : 0 : 143,920,350:2%
2023-05-27 04:07:33,607 - INFO: 2013-03-09 17:28:51 : 1,200,000 : 171 : 0 : 143,920,350:2%
2023-05-27 04:07:33,607 - INFO: 2013-03-09 17:28:51 : 1,200,000 : 171 : 0 : 143,920,350:2%
2023-05-27 04:07:33,607 - INFO: 2013-03-09 17:28:51 : 1,200,000 : 171 : 0 : 143,920,350:2%
2023-05-27 04:07:33,607 - INFO: 2013-03-09 17:28:51 : 1,200,000 : 171 : 0 : 143,920,350:2%
2023-05-27 04:07:33,607 - INFO: 2013-03-09 17:28:51 : 1,200,000 : 171 : 0 : 143,920,350:2%

2023-05-27 04:13:29,742 - INFO: 2013-06-24 05:19:26 : 1,800,000 : 171 : 0 : 211,817,200:3%
2023-05-27 04:13:29,742 - INFO: 2013-06-24 05:19:26 : 1,800,000 : 171 : 0 : 211,817,200:3%
2023-05-27 04:13:29,742 - INFO: 2013-06-24 05:19:26 : 1,800,000 : 171 : 0 : 211,817,200:3%
2023-05-27 04:13:29,742 - INFO: 2013-06-24 05:19:26 : 1,800,000 : 171 : 0 : 211,817,200:3%
2023-05-27 04:13:29,742 - INFO: 2013-06-24 05:19:26 : 1,800,000 : 171 : 0 : 211,817,200:3%
2023-05-27 04:13:29,742 - INFO: 2013-06-24 05:19:26 : 1,800,000 : 171 : 0 : 211,817,200:3%
2023-05-27 04:13:29,742 - INFO: 2013-06-24 05:19:26 : 1,800,000 : 171 : 0 : 211,817,200:3%
2023-05-27 04:13:29,742 - INFO: 2013-06-24 05:19:26 : 1,800,000 : 171 : 0 : 211,817,200:3%
2023-05-27 04:13:29,742 - INFO: 2013-06-24 05:19:26 : 1,800,000 : 171 : 0 : 211,817,200:3%
2023-05-27 04:13:29,742 - INFO: 2013-06-24 05:19:26 : 1,800,000 : 171 : 0 : 211,817,200:3%
2023-05-27 04:13:29,742 - INFO: 2013-06-24 05:19:26 : 1,800,000 : 171 : 0 : 211,817,200:3%

2023-05-27 04:19:26,576 - INFO: 2013-09-19 23:41:13 : 2,400,000 : 287 : 0 : 259,135,275:3%
2023-05-27 04:19:26,576 - INFO: 2013-09-19 23:41:13 : 2,400,000 : 287 : 0 : 259,135,275:3%
2023-05-27 04:19:26,576 - INFO: 2013-09-19 23:41:13 : 2,400,000 : 287 : 0 : 259,135,275:3%
2023-05-27 04:19:26,576 - INFO: 2013-09-19 23:41:13 : 2,400,000 : 287 : 0 : 259,135,275:3%
2023-05-27 04:19:26,576 - INFO: 2013-09-19 23:41:13 : 2,400,000 : 287 : 0 : 259,135,275:3%
2023-05-27 04:19:26,576 - INFO: 2013-09-19 23:41:13 : 2,400,000 : 287 : 0 : 259,135,275:3%
2023-05-27 04:19:26,576 - INFO: 2013-09-19 23:41:13 : 2,400,000 : 287 : 0 : 259,135,275:3%
2023-05-27 04:19:26,576 - INFO: 2013-09-19 23:41:13 : 2,400,000 : 287 : 0 : 259,135,275:3%
2023-05-27 04:19:26,576 - INFO: 2013-09-19 23:41:13 : 2,400,000 : 287 : 0 : 259,135,275:3%
2023-05-27 04:19:26,576 - INFO: 2013-09-19 23:41:13 : 2,400,000 : 287 : 0 : 259,135,275:3%
2023-05-27 04:20:26,424 - INFO: 2013-10-04 20:40:29 : 2,500,000 : 287 : 0 : 282,859,850:4%

...

2023-05-27 16:53:47,096 - INFO: 2022-09-08 12:28:54 : 77,100,000 : 23,459 : 0 : 7,621,355,875:97%
2023-05-27 16:53:47,096 - INFO: 2022-09-08 12:28:54 : 77,100,000 : 23,459 : 0 : 7,621,355,875:97%
2023-05-27 16:54:46,990 - INFO: 2022-09-12 17:19:49 : 77,200,000 : 23,459 : 0 : 7,631,055,425:97%
2023-05-27 16:54:46,990 - INFO: 2022-09-12 17:19:49 : 77,200,000 : 23,459 : 0 : 7,631,055,425:97%
2023-05-27 16:54:46,990 - INFO: 2022-09-12 17:19:49 : 77,200,000 : 23,459 : 0 : 7,631,055,425:97%
2023-05-27 16:54:46,990 - INFO: 2022-09-12 17:19:49 : 77,200,000 : 23,459 : 0 : 7,631,055,425:97%
2023-05-27 16:54:46,990 - INFO: 2022-09-12 17:19:49 : 77,200,000 : 23,459 : 0 : 7,631,055,425:97%
2023-05-27 16:54:46,990 - INFO: 2022-09-12 17:19:49 : 77,200,000 : 23,459 : 0 : 7,631,055,425:97%
2023-05-27 16:54:46,990 - INFO: 2022-09-12 17:19:49 : 77,200,000 : 23,459 : 0 : 7,631,055,425:97%
2023-05-27 16:54:46,990 - INFO: 2022-09-12 17:19:49 : 77,200,000 : 23,459 : 0 : 7,631,055,425:97%
2023-05-27 16:54:46,

2023-05-27 16:59:25,871 - INFO: 2022-10-07 05:05:50 : 77,700,000 : 23,565 : 0 : 7,669,460,400:98%
2023-05-27 16:59:25,871 - INFO: 2022-10-07 05:05:50 : 77,700,000 : 23,565 : 0 : 7,669,460,400:98%
2023-05-27 16:59:25,871 - INFO: 2022-10-07 05:05:50 : 77,700,000 : 23,565 : 0 : 7,669,460,400:98%
2023-05-27 16:59:25,871 - INFO: 2022-10-07 05:05:50 : 77,700,000 : 23,565 : 0 : 7,669,460,400:98%
2023-05-27 16:59:25,871 - INFO: 2022-10-07 05:05:50 : 77,700,000 : 23,565 : 0 : 7,669,460,400:98%
2023-05-27 16:59:25,871 - INFO: 2022-10-07 05:05:50 : 77,700,000 : 23,565 : 0 : 7,669,460,400:98%
2023-05-27 16:59:25,871 - INFO: 2022-10-07 05:05:50 : 77,700,000 : 23,565 : 0 : 7,669,460,400:98%
2023-05-27 16:59:25,871 - INFO: 2022-10-07 05:05:50 : 77,700,000 : 23,565 : 0 : 7,669,460,400:98%
2023-05-27 17:00:21,549 - INFO: 2022-10-12 12:26:55 : 77,800,000 : 23,567 : 0 : 7,678,897,800:98%
2023-05-27 17:00:21,549 - INFO: 2022-10-12 12:26:55 : 77,800,000 : 23,567 : 0 : 7,678,897,800:98%
2023-05-27 17:00:21,

2023-05-27 17:05:04,950 - INFO: 2022-11-02 16:57:28 : 78,300,000 : 23,568 : 0 : 7,725,560,500:98%
2023-05-27 17:05:04,950 - INFO: 2022-11-02 16:57:28 : 78,300,000 : 23,568 : 0 : 7,725,560,500:98%
2023-05-27 17:05:04,950 - INFO: 2022-11-02 16:57:28 : 78,300,000 : 23,568 : 0 : 7,725,560,500:98%
2023-05-27 17:05:04,950 - INFO: 2022-11-02 16:57:28 : 78,300,000 : 23,568 : 0 : 7,725,560,500:98%
2023-05-27 17:05:04,950 - INFO: 2022-11-02 16:57:28 : 78,300,000 : 23,568 : 0 : 7,725,560,500:98%
2023-05-27 17:05:04,950 - INFO: 2022-11-02 16:57:28 : 78,300,000 : 23,568 : 0 : 7,725,560,500:98%
2023-05-27 17:05:04,950 - INFO: 2022-11-02 16:57:28 : 78,300,000 : 23,568 : 0 : 7,725,560,500:98%
2023-05-27 17:05:04,950 - INFO: 2022-11-02 16:57:28 : 78,300,000 : 23,568 : 0 : 7,725,560,500:98%
2023-05-27 17:05:04,950 - INFO: 2022-11-02 16:57:28 : 78,300,000 : 23,568 : 0 : 7,725,560,500:98%
2023-05-27 17:05:04,950 - INFO: 2022-11-02 16:57:28 : 78,300,000 : 23,568 : 0 : 7,725,560,500:98%
2023-05-27 17:05:04,

2023-05-27 17:09:45,126 - INFO: 2022-11-21 22:53:49 : 78,800,000 : 23,886 : 0 : 7,771,698,900:99%
2023-05-27 17:09:45,126 - INFO: 2022-11-21 22:53:49 : 78,800,000 : 23,886 : 0 : 7,771,698,900:99%
2023-05-27 17:09:45,126 - INFO: 2022-11-21 22:53:49 : 78,800,000 : 23,886 : 0 : 7,771,698,900:99%
2023-05-27 17:09:45,126 - INFO: 2022-11-21 22:53:49 : 78,800,000 : 23,886 : 0 : 7,771,698,900:99%
2023-05-27 17:09:45,126 - INFO: 2022-11-21 22:53:49 : 78,800,000 : 23,886 : 0 : 7,771,698,900:99%
2023-05-27 17:10:41,486 - INFO: 2022-11-25 19:00:43 : 78,900,000 : 23,887 : 0 : 7,781,136,300:99%
2023-05-27 17:10:41,486 - INFO: 2022-11-25 19:00:43 : 78,900,000 : 23,887 : 0 : 7,781,136,300:99%
2023-05-27 17:10:41,486 - INFO: 2022-11-25 19:00:43 : 78,900,000 : 23,887 : 0 : 7,781,136,300:99%
2023-05-27 17:10:41,486 - INFO: 2022-11-25 19:00:43 : 78,900,000 : 23,887 : 0 : 7,781,136,300:99%
2023-05-27 17:10:41,486 - INFO: 2022-11-25 19:00:43 : 78,900,000 : 23,887 : 0 : 7,781,136,300:99%
2023-05-27 17:10:41,

2023-05-27 17:15:23,297 - INFO: 2022-12-16 01:25:47 : 79,400,000 : 24,138 : 0 : 7,828,585,450:100%
2023-05-27 17:15:23,297 - INFO: 2022-12-16 01:25:47 : 79,400,000 : 24,138 : 0 : 7,828,585,450:100%
2023-05-27 17:15:23,297 - INFO: 2022-12-16 01:25:47 : 79,400,000 : 24,138 : 0 : 7,828,585,450:100%
2023-05-27 17:15:23,297 - INFO: 2022-12-16 01:25:47 : 79,400,000 : 24,138 : 0 : 7,828,585,450:100%
2023-05-27 17:15:23,297 - INFO: 2022-12-16 01:25:47 : 79,400,000 : 24,138 : 0 : 7,828,585,450:100%
2023-05-27 17:15:23,297 - INFO: 2022-12-16 01:25:47 : 79,400,000 : 24,138 : 0 : 7,828,585,450:100%
2023-05-27 17:15:23,297 - INFO: 2022-12-16 01:25:47 : 79,400,000 : 24,138 : 0 : 7,828,585,450:100%
2023-05-27 17:15:23,297 - INFO: 2022-12-16 01:25:47 : 79,400,000 : 24,138 : 0 : 7,828,585,450:100%
2023-05-27 17:15:23,297 - INFO: 2022-12-16 01:25:47 : 79,400,000 : 24,138 : 0 : 7,828,585,450:100%
2023-05-27 17:15:23,297 - INFO: 2022-12-16 01:25:47 : 79,400,000 : 24,138 : 0 : 7,828,585,450:100%
2023-05-27

In [75]:
# Loading the extracted file: 
filtered_comments = pd.read_csv('news_comments_id_filtered.csv', header=None)

In [76]:
filtered_comments.head()

,0,1,2
0,t3_flgxp,c1gtf8n,I don't advocate the death penalty. But this ...
1,t3_flgxp,c1gtjcm,Did anybody else see [this video](http://crook...
2,t3_flgxp,c1gtlbj,I've been following this case for a while. I'm...
3,t3_flgxp,c1gtmdw,Look at her eyes in that photo. She's not cra...
4,t3_flgxp,c1gtnqs,Good.


In [77]:
# adding a header column 
header = ["post_id", "comment_id", "comment"]
filtered_comments.columns = header

In [78]:
filtered_comments.head()

,post_id,comment_id,comment
0,t3_flgxp,c1gtf8n,I don't advocate the death penalty. But this ...
1,t3_flgxp,c1gtjcm,Did anybody else see [this video](http://crook...
2,t3_flgxp,c1gtlbj,I've been following this case for a while. I'm...
3,t3_flgxp,c1gtmdw,Look at her eyes in that photo. She's not cra...
4,t3_flgxp,c1gtnqs,Good.


In [79]:
# remove t3_
filtered_comments["post_id"] = filtered_comments["post_id"].str.replace(r"^t3_", "", regex =True)

In [82]:
print(f"N={len(filtered_comments)} replies to the posts(i.e., top-level comments) were extracted for the random sample of N={len(posts_red)} reddit posts about immigration")

N=24149 replies to the posts(i.e., top-level comments) were extracted for the random sample of N=6000 reddit posts about immigration


# Basic Data Removal 
- More extensive data cleaning will be done seperately on the data for the baseline classifiers. However, for both baseline and transformer, duplicates should be removed and "[deleted]" or "[removed]" posts or comments are removed from the data to ensure the samples for labelling, and the unlabelled dataset contain meaningful data.

In [112]:
# function for basic preprocessing to remove comments with deleted or removed in the text body, and to drop 
# NAs and duplicates
def base_preprocessing(df, column):
    df = df[~df[column].str.contains(r"\[deleted\]|\[removed\]", regex=True)]
    df = df.dropna().drop_duplicates()
    return df

In [114]:
# apply function to filtered comments
filtered_comments_pre = base_preprocessing(filtered_comments, "comment")

In [115]:
len(filtered_comments_pre)

16518

In [116]:
# apply preprocessing function to posts
red_posts_pre = base_preprocessing(posts_red, "title")

In [117]:
len(red_posts_pre)

6000

# Create sample for labelling, set aside test dataset

To get the dataframes for labelling, a random sample of N=1000 will be drawn from the posts and comments each. These are then removed from the original dataframes, generating the unlabelled files. 

### Posts for coding moralised frames:

In [118]:
# draw 1000 random posts and save for labelling, setting random state so same data is drawn when rerunning script
labelled_posts = red_posts_pre.sample(1000, random_state=100)

In [119]:
# save labelled data to csv 
labelled_posts.to_csv("labelled_posts_new.csv")

In [120]:
# remove all posts to be labelled from the original dataframe to get unlabelled dataframe
unlabelled_posts = red_posts_pre.drop(labelled_posts.index)

In [121]:
# save unlabelled posts to csv
unlabelled_posts.to_csv("unlabelled_posts_new.csv")

### Comments for coding language that indicates affective polarisation

In [122]:
# draw 1000 random comments and save for labelling, setting random state so same data is drawn when rerunning script
labelled_comments = filtered_comments_pre.sample(1000, random_state=100)

In [123]:
# save labelled data to csv 
labelled_comments.to_csv("labelled_comments.csv")

In [124]:
# remove all comments to be labelled from the dataframe to get unlabelled data
unlabelled_comments = filtered_comments_pre.drop(labelled_comments.index)

In [125]:
# save unlabelled posts to csv
unlabelled_comments.to_csv("unlabelled_comments.csv")